In [1]:
import numpy  as np
import sympy  as sym

import matplotlib.pyplot as plt
import scipy.sparse      as sparse

import utilitis_FEEC.evaluation as eva
import utilitis_FEEC.bsplines   as bsp

import utilitis_FEEC.projections_mhd as mhd
import utilitis_FEEC.projectors_opt as proj

In [2]:
# ... define boundary conditions in each direction (True: periodic, False: else)
bc = [False, False, False]
bc_1, bc_2, bc_3 = bc


# ... number of elements and element boundaries in each direction
Nel_1 = 5
Nel_2 = 5
Nel_3 = 5

el_b_1 = np.linspace(0., 1., Nel_1 + 1)
el_b_2 = np.linspace(0., 1., Nel_2 + 1)
el_b_3 = np.linspace(0., 1., Nel_3 + 1)

# ... degree of spline basis (of the 0-forms) in each direction
p = [3, 3, 3]
p1, p2, p3 = p


# ... geometry
mapping = 'cube'


# ... number of basis functions
Nbase = [Nel_1 + p1, Nel_2 + p2, Nel_3 + p3] 
Nbase_1, Nbase_2, Nbase_3 = Nbase


# ... knot vectors
T1 = bsp.make_knots(el_b_1, p1, bc_1)
T2 = bsp.make_knots(el_b_2, p2, bc_2)
T3 = bsp.make_knots(el_b_3, p3, bc_3)    
T = [T1, T2, T3]



# ... define physical domain and mapping from logical domain
if mapping == 'annulus':
    
    # ... coordinates
    r, phi, z = sym.symbols('r, phi, z')
    q = sym.Matrix([r, phi, z])
    
    # ... mapping
    R1 = 0.2         # inner radius
    R2 = 1.0         # outer radius
    dR = R2 - R1     # thickness
    
    F = sym.Matrix([(r*dR + R1)*sym.cos(2*sym.pi*phi), (r*dR + R1)*sym.sin(2*sym.pi*phi), z])
    
elif mapping == 'torus':
    
    # ... coordinates
    r, theta, phi = sym.symbols('r, theta, phi')
    q = sym.Matrix([r, theta, phi])

    # ... mapping
    R0 = 1.5         # major radius
    R1 = 0.2         # inner radius
    R2 = 1.0         # outer radius
    dR = R2 - R1     # thickness
    
    F = sym.Matrix([(R0 + (r*dR + R1)*sym.cos(2*sym.pi*theta))*sym.cos(2*sym.pi*phi), (R0 + (r*dR + R1)*sym.cos(2*sym.pi*theta))*sym.sin(2*sym.pi*phi), (r*dR + R1)*sym.sin(2*sym.pi*theta)])
    
elif mapping == 'cube':
    
    # ... coordinates
    x, y, z = sym.symbols('x, y, z')
    q = sym.Matrix([x, y, z])
    
    # ... mapping
    Lx = 1.           # length in x
    Ly = 1.           # length in y
    Lz = 1.           # length in z
    
    F = sym.Matrix([Lx*x, Ly*y, Lz*z])
    

# ... jacobian matrix
DF = F.jacobian(q)

# ... metric tensor
G = sym.simplify(DF.transpose()*DF)

# ... inverse of metric tensor
Ginv = G.inverse()

#... square root of jacobi determinant
g = sym.simplify(G.det())
g_sqrt = sym.sqrt(g)


# ... convert mapping functions to callables
xc = sym.lambdify(q, F[0])
yc = sym.lambdify(q, F[1])
zc = sym.lambdify(q, F[2])

G = [[sym.lambdify(q, G[0, 0]), sym.lambdify(q, G[0, 1]), sym.lambdify(q, G[0, 2])], [sym.lambdify(q, G[1, 0]), sym.lambdify(q, G[1, 1]), sym.lambdify(q, G[1, 2])], [sym.lambdify(q, G[2, 0]), sym.lambdify(q, G[2, 1]), sym.lambdify(q, G[2, 2])]]
Ginv = [[sym.lambdify(q, Ginv[0, 0]), sym.lambdify(q, Ginv[0, 1]), sym.lambdify(q, Ginv[0, 2])], [sym.lambdify(q, Ginv[1, 0]), sym.lambdify(q, Ginv[1, 1]), sym.lambdify(q, Ginv[1, 2])], [sym.lambdify(q, Ginv[2, 0]), sym.lambdify(q, Ginv[2, 1]), sym.lambdify(q, Ginv[2, 2])]]
g = sym.lambdify(q, g)
g_sqrt = sym.lambdify(q, g_sqrt)

In [3]:
MHD = mhd.projections_mhd(p, Nbase, T, bc)

In [4]:
PRO = proj.projectors_3d(p, Nbase, T, bc)

PRO.assemble_V1()
PRO.assemble_V2()

In [5]:
rho0 = lambda q1, q2, q3 : q1 * (1 - q2) * q3

uini_1 = lambda q1, q2, q3 : q1*np.sin(2*np.pi*q1) * q2**2*np.sin(2*np.pi*q2) * q3**3*np.sin(2*np.pi*q3)
uini_2 = lambda q1, q2, q3 : q1**3*np.sin(2*np.pi*q1) * q2*np.sin(2*np.pi*q2) * q3**2*np.sin(2*np.pi*q3)
uini_3 = lambda q1, q2, q3 : q1**2*np.sin(2*np.pi*q1) * q2**3*np.sin(2*np.pi*q2) * q3*np.sin(2*np.pi*q3)

In [6]:
uvec = PRO.PI_1([uini_1, uini_2, uini_3])

In [7]:
MHD.assemble_equilibrium(rho0, Ginv)

In [8]:
test1, test2, test3 = MHD.rhs_A(uvec)